MTG Card Identifier  

In [25]:
import requests
import pandas as pd
import sys


def fetch_card_data(card_name: str) -> dict:
    url = f"https://api.scryfall.com/cards/named?exact={card_name}"
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()
    return {
        "Name": data.get("name"),
        "Set": data.get("set_name"),
        "Rarity": data.get("rarity"),
        "Oracle Text": data.get("oracle_text"),
        "Image URL": data.get("image_uris", {}).get("normal"),
        "Price USD": data.get("prices", {}).get("usd")
    }

def get_usd_to_eur_rate() -> float:
    """
    Holt den aktuellen USD→EUR-Kurs von frankfurter.app (ohne API-Key).
    """
    url = "https://api.frankfurter.app/latest"
    params = {"from": "USD", "to": "EUR"}
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    # data == {"amount":1,"base":"USD","date":"2025-06-08","rates":{"EUR":0.9064}}
    return data["rates"]["EUR"]

# --- Testaufruf ---
sample = "lightning bolt"
card_info = fetch_card_data(sample)
df = pd.DataFrame([card_info])

# Wechselkurs holen
rate = get_usd_to_eur_rate()
print(f"Aktueller USD→EUR-Kurs: {rate:.4f}")

# Konvertieren und neue Spalte setzen
try:
    usd = float(card_info["Price USD"])
    eur = round(usd * rate, 2)
    df["Price EUR"] = eur
except (TypeError, ValueError):
    df["Price EUR"] = None
    print("Warnung: Kein gültiger USD-Preis vorhanden, EUR-Spalte leer.")

# Ergebnis anzeigen & als CSV speichern
display(df)

# filename = sample.replace(" ", "_") + "_info.csv"
# df.to_csv(filename, index=False)
# print(f"CSV-Datei gespeichert unter: {filename}")


Aktueller USD→EUR-Kurs: 0.8763


,Name,Set,Rarity,Oracle Text,Image URL,Price USD,Price EUR
0,Lightning Bolt,Ravnica: Clue Edition,uncommon,Lightning Bolt deals 3 damage to any target.,https://cards.scryfall.io/normal/front/7/7/77c...,0.86,0.75


In [26]:
# 2.2 Feinschliff: Error-Handling + Spalten-Format + CSV-Export

from IPython.display import display
import pandas as pd

# Kartennamen hier setzen (später via CLI in .py):
sample = "Lightning Bolt"

try:
    # Daten holen
    card_info = fetch_card_data(sample)
    
    # DataFrame bauen
    df = pd.DataFrame([card_info])
    
    # Kurs holen und umrechnen
    rate = get_usd_to_eur_rate()
    try:
        usd = float(card_info["Price USD"])
        eur = round(usd * rate, 2)
    except (TypeError, ValueError):
        eur = None
    
    # Spalten ergänzen und auswählen
    df["Price EUR"] = eur
    cols = ["Name", "Set", "Rarity", "Price USD", "Price EUR"]
    df = df[cols]
    
    # Ausgabe
    print(f"Aktueller USD→EUR-Kurs: {rate:.4f}")
    display(df)

    # CSV speichern
    filename = sample.replace(" ", "_") + "_info.csv"
    df.to_csv(filename, index=False)
    print(f"✅ CSV gespeichert: {filename}")

except requests.HTTPError as e:
    print(f"❌ Karte nicht gefunden oder API-Fehler ({e.response.status_code})")
except Exception as e:
    print(f"❌ Unerwarteter Fehler: {e}")


Aktueller USD→EUR-Kurs: 0.8763


,Name,Set,Rarity,Price USD,Price EUR
0,Lightning Bolt,Ravnica: Clue Edition,uncommon,0.86,0.75


✅ CSV gespeichert: Lightning_Bolt_info.csv
